In [1]:
from mnist import MNIST
import numpy as np, scipy, seaborn as sns
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier

ImportError: No module named mnist

In [ ]:
mndata = MNIST('../data')
TrainingData = mndata.load_training()
TestData     = mndata.load_testing()

In [ ]:
X_train = np.array(TrainingData[0])
Y_train = np.array(TrainingData[1])
X_test  = np.array(TestData[0])
Y_test  = np.array(TestData[1])

In [ ]:
def Visualize(a):
    data = a.reshape((28, 28))
    imshow(data)
Visualize(X_train[333])
print "the number shoud be", Y_train[333]

In [ ]:
X_std = StandardScaler().fit_transform(X_train)

Test sklearn.decomposition.PCA

In [ ]:
def printc(cc):
    sns.palplot(cc)
    print "     " + "      ".join(str(w) for w in xrange(10))

def minist_plot_2d(x, y, label=Y_train, num = 10000, fs=(10, 10), cc = sns.color_palette("Set3", n_colors=10, desat=1)):
    printc(cc)
    fig = plt.figure(figsize=fs)
    plt.scatter(x[:num], y[:num], c=[cc[w] for w in label[:num]], marker='o', s = 7)

def minist_plot_3d(x, y, z, label=Y_train, num = 10000, fs=(10, 10), cc = sns.color_palette("Set3", n_colors=10, desat=1)):
    printc(cc)
    fig = plt.figure(figsize=fs)
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x[:num], y[:num], z[:num], c=[cc[w] for w in label[:num]], marker='o', s = 7)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(X_std.T)

In [ ]:
pca_score = pca.explained_variance_ratio_
V = pca.components_
print pca_score, V.shape

In [ ]:
minist_plot_3d(V[0], V[1], V[2], num=30000)

In [ ]:
pca = PCA(n_components=2)
pca.fit(X_std.T)
pca_score = pca.explained_variance_ratio_
V = pca.components_

In [ ]:
minist_plot_2d(V[0], V[1], num=30000)

Test SVD PCA

In [ ]:
u,s,v = scipy.sparse.linalg.svds(X_std, k=3)
print u.shape, s.shape, v.shape

In [ ]:
minist_plot_3d(u.T[0][:num], u.T[1][:num], u.T[2][:num], num=30000)

In [ ]:
u,s,v = scipy.sparse.linalg.svds(X_std, k=2)
print u.shape, s.shape, v.shape

In [ ]:
minist_plot_2d(u.T[0][:num], u.T[1][:num], num=30000)

Normal Approach

In [ ]:
cov_mat = np.cov(X_std.T)
print('NumPy covariance matrix: \n%s' %cov_mat)

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

temp = sorted([(eig_vals[i], eig_vecs[i]) for i in xrange(len(eig_vals))], key = lambda x: abs(x[0]), reverse = True)
eig_vals = np.array([w[0] for w in temp])
eig_vecs = np.array([w[1] for w in temp])

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

In [ ]:
eig_vecs.shape

In [ ]:
X_transformed = X_std.dot(eig_vecs.T[:3].T)

In [ ]:
minist_plot_3d(X_transformed[:num, 0], X_transformed[:num, 1], X_transformed[:num, 2], num=30000)

In [ ]:
X_transformed = X_std.dot(eig_vecs.T[:2].T)

In [ ]:
minist_plot_2d(X_transformed[:num, 0], X_transformed[:num, 1], num=30000)

In [ ]:
model = TSNE(n_components=3, random_state=0)
np.set_printoptions(suppress=True)
embed = model.fit_transform(X_std[:10000])

In [ ]:
minist_plot_3d(embed[:,0], embed[:,1], embed[:,2])

In [ ]:
model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
embed = model.fit_transform(X_std[:10000])

In [ ]:
minist_plot_2d(embed[:,0], embed[:,1])

PCA-KNN Part

In [ ]:
def pca_cov(X, n=3):
    # Input X should be instance x feature_size
    u,s,v = scipy.sparse.linalg.svds(X_std, k=n)
    # Results are still instance x new_feature_size(n)
    return u

In [ ]:
X_all = np.concatenate([X_train, X_test])
X_std = StandardScaler().fit_transform(X_all)

In [ ]:
def test_pca(d = 40):
    X_all_now = pca_cov(X_std, n = d)
    X_train_now = X_all_now[:len(X_train)]
    X_test_now = X_all_now[-len(X_test):]
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train_now, Y_train)
    Results = knn.predict(X_test_now)
    count = 0
    for i in xrange(len(Results)):
        if (Results[i] == Y_test[i]): count += 1
    print 'Accuracy:', count * 100 / float(len(Results)), '%'

In [ ]:
test_pca(40)

In [ ]:
test_pca(80)

In [ ]:
test_pca(200)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=334)
pca.fit(X_std)
print np.sum(pca.explained_variance_ratio_) 

In [ ]:
test_pca(334)